<a href="https://colab.research.google.com/github/ksetdekov/trip_choice_optimizer/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

from  scipy.stats import gamma, norm

In [2]:
# np.random.seed(0) # фиксируем seed

# risk tolerance
$$\rho = 3.37$$

# initialize

In [3]:
mu_hat_i = 0 # mean
t_i = 0
alpha_i = 0.5
beta_i = 0.5
# rho = 3.37
rho = 1

In [4]:
def update_mean(X, T_last, mu_last):
    mu_new = T_last * mu_last / (T_last + 1) + X / (T_last + 1)
    return mu_new

def update_samples(T):
    return T + 1

def update_shape(a):
    return a + 0.5

def update_rate(X, mu_last, beta_last, T_last):
    beta_new = beta_last + (T_last / (T_last + 1)) * (np.square(X - mu_last)) / 2
    return beta_new

In [5]:
hands = pd.DataFrame({'name': ['трамвай + 944', 'с799 + пешком', 'пешком + 944', 'мотоцикл', 'с799 + 944', 
                               'рыжая + жёлтая + салатовая', 'трамвай + салатовая',
                               'каршеринг', 'серая + салатовая', 'рыжая + фиолетовая + салатовая', 'рыжая + салатовая',
                               'машина', 'самокат', '2гис', 'яндекс карты', 'гугл карты', 'citymapper', 'транспорт москвы']})

# 'московский велосипед'

In [6]:
hands['mu'] = 0
hands['Te'] = 0
hands['alpha'] = 0.5
hands['beta'] = 0.5

hands

,name,mu,Te,alpha,beta
0,трамвай + 944,0,0,0.5,0.5
1,с799 + пешком,0,0,0.5,0.5
2,пешком + 944,0,0,0.5,0.5
3,мотоцикл,0,0,0.5,0.5
4,с799 + 944,0,0,0.5,0.5
5,рыжая + жёлтая + салатовая,0,0,0.5,0.5
6,трамвай + салатовая,0,0,0.5,0.5
7,каршеринг,0,0,0.5,0.5
8,серая + салатовая,0,0,0.5,0.5
9,рыжая + фиолетовая + салатовая,0,0,0.5,0.5


In [7]:
def to_minutes(timestr: str):
  '''
  convert timestr to float minutes
  '''
  return pd.to_timedelta(timestr).total_seconds()/60

In [8]:
def update_hands(hands, name, value):
    if isinstance(value, str):
        try:
            value = to_minutes(value)
        except ValueError:
            raise ValueError('input time string in hh:mm:ss format')
    elif isinstance(value, float) or isinstance(value, int):
        pass
    else:
        raise ValueError('input time string or int/float value')
    # print(value)
    _, mu, t, alpha, beta = hands[hands.name ==name].values[0]
    beta = update_rate(value, mu, beta, t)
    mu =  update_mean(value, t, mu)
    t = update_samples(t)
    alpha = update_shape(alpha)

    # print(mu, t, alpha, beta)

    hands.loc[hands.name ==name, 'mu'] = mu
    hands.loc[hands.name ==name, 'Te'] = t
    hands.loc[hands.name ==name, 'alpha'] = alpha
    hands.loc[hands.name ==name, 'beta'] = beta

    return hands


In [9]:
hands = update_hands(hands, 'мотоцикл', 20)
hands = update_hands(hands, 'рыжая + жёлтая + салатовая', 47.63)
hands = update_hands(hands, 'каршеринг', 31.75)
hands = update_hands(hands, 'рыжая + салатовая', 52.6)
hands = update_hands(hands, 'каршеринг', 23.15)
hands = update_hands(hands, 'рыжая + фиолетовая + салатовая', 52.82)
hands = update_hands(hands, 'серая + салатовая', 52.37)
hands = update_hands(hands, 'мотоцикл', 21.05)
hands = update_hands(hands, 'мотоцикл', 26.40)
hands = update_hands(hands, 'мотоцикл', "0:25:18")
hands = update_hands(hands, 'мотоцикл', "0:24:54")
hands = update_hands(hands, 'мотоцикл', "0:26:51")
hands = update_hands(hands, 'каршеринг', "0:28:04")
hands = update_hands(hands, 'каршеринг', "0:28:05")
hands = update_hands(hands, 'мотоцикл', "0:24:54")
hands = update_hands(hands, 'мотоцикл', "0:26:58")
hands = update_hands(hands, 'мотоцикл', "0:26:29")
hands = update_hands(hands, 'машина', "0:23:59")
hands = update_hands(hands, 'каршеринг', "0:36:41")
hands = update_hands(hands, 'машина', "0:24:49")
hands = update_hands(hands, 'машина', "0:21:56")
hands = update_hands(hands, 'машина', "0:22:03")
hands = update_hands(hands, 'машина', "0:23:21")
hands = update_hands(hands, 'мотоцикл', "0:27:45")
hands = update_hands(hands, 'машина', "0:22:26")
hands = update_hands(hands, 'машина', "0:25:00")
hands = update_hands(hands, 'машина', "0:24:07")
hands = update_hands(hands, 'машина', "0:23:38")
hands = update_hands(hands, 'машина', "0:24:13")
hands = update_hands(hands, 'машина', "0:25:32")
hands = update_hands(hands, 'машина', "0:22:11")
hands = update_hands(hands, 'самокат', "0:47:17")
hands = update_hands(hands, 'машина', "0:23:33")
hands = update_hands(hands, 'машина', "0:22:54")
hands = update_hands(hands, 'машина', "0:22:42")
hands = update_hands(hands, 'каршеринг', "0:33:02")
hands = update_hands(hands, 'машина', "0:23:31")
hands = update_hands(hands, 'машина', "0:24:39")
hands = update_hands(hands, 'машина', "0:24:16")
hands = update_hands(hands, 'машина', "0:28:12")
hands = update_hands(hands, 'трамвай + 944', "0:41:35")
hands = update_hands(hands, 'машина', "0:26:27")
hands = update_hands(hands, 'машина', "0:23:55")
hands = update_hands(hands, 'с799 + 944', "0:45:48")
hands = update_hands(hands, 'машина', "0:32:23")
hands = update_hands(hands, 'машина', "0:22:34")
hands = update_hands(hands, 'машина', "0:25:00")
hands = update_hands(hands, 'машина', "0:27:01")
hands = update_hands(hands, 'каршеринг', "0:31:14")
hands = update_hands(hands, 'трамвай + салатовая', "0:48:40")
hands = update_hands(hands, 'пешком + 944', "0:46:56")
hands = update_hands(hands, 'машина', "0:31:17")
hands = update_hands(hands, 'каршеринг', "0:34:08")
hands = update_hands(hands, 'машина', "0:31:18")
hands = update_hands(hands, 'машина', "0:27:10")
hands = update_hands(hands, 'каршеринг', "0:25:51")
hands = update_hands(hands, 'транспорт москвы', "0:49:36")
hands = update_hands(hands, 'яндекс карты', "0:44:21")


hands

,name,mu,Te,alpha,beta
0,трамвай + 944,41.583333,1,1.0,0.500000
1,с799 + пешком,0.000000,0,0.5,0.500000
2,пешком + 944,46.933333,1,1.0,0.500000
3,мотоцикл,25.060000,10,5.5,30.344778
4,с799 + 944,45.800000,1,1.0,0.500000
5,рыжая + жёлтая + салатовая,47.630000,1,1.0,0.500000
6,трамвай + салатовая,48.666667,1,1.0,0.500000
7,каршеринг,30.220370,9,5.0,73.827716
8,серая + салатовая,52.370000,1,1.0,0.500000
9,рыжая + фиолетовая + салатовая,52.820000,1,1.0,0.500000


5: for each t = K + 1, K + 2, . . ., do

7: Sample τi,t from Gamma(αi,t−1, βi,t−1).

8: Sample θi,t from N (ˆµi,t−1, 1/Ti,t−1)

9: Play arm i(t) = arg maxi∈[K] ρθi,t − 1/τi,t and observe reward Xi(t),t

10: Update(ˆµi(t),t−1, Ti(t),t−1, αi(t),t−1, βi(t),t−1)

11: end for


In [10]:
tau = gamma.rvs( a = hands.alpha, scale=1/hands.beta)
theta = norm.rvs(hands.mu, 1/hands.Te)
hands['tau'] = tau
hands['theta'] = theta
hands['SD'] = np.sqrt(1/tau)
hands['var95'] = theta + norm.ppf(1-0.05/2) * hands.SD

if hands.mu.min() == 0:
    output1 = hands.reindex(np.argsort(hands.Te))
else:
    output1 = hands.reindex(np.argsort(rho * theta + 1/tau))

output1

,name,mu,Te,alpha,beta,tau,theta,SD,var95
Te,,,,,,,,,
1,с799 + пешком,0.000000,0,0.5,0.500000,0.197111,-inf,2.252393,-inf
15,гугл карты,0.000000,0,0.5,0.500000,8.448886,inf,0.344033,inf
13,2гис,0.000000,0,0.5,0.500000,1.141627,-inf,0.935918,-inf
16,citymapper,0.000000,0,0.5,0.500000,5.532104,-inf,0.425162,-inf
0,трамвай + 944,41.583333,1,1.0,0.500000,0.108486,41.509790,3.036082,47.460402
14,яндекс карты,44.350000,1,1.0,0.500000,3.094220,44.662621,0.568492,45.776845
12,самокат,47.283333,1,1.0,0.500000,0.249963,48.639353,2.000147,52.559570
10,рыжая + салатовая,52.600000,1,1.0,0.500000,2.160432,52.159924,0.680346,53.493378
9,рыжая + фиолетовая + салатовая,52.820000,1,1.0,0.500000,3.578862,54.535100,0.528600,55.571137


## для хилокомода

In [11]:
hands_drops = pd.DataFrame({'name': ['хилокомод', 'гилан', 'гилан ультра комфорт', 'стиллавит', 'артелак баланс', 
                                     'артелак всплеск уно', 'артелак всплеск', 'систейн ультра плюс', 'визмед', 'визмед лайт', 
                                     'хилабак', 'окутиарз'
                                     ]})
hands_drops['mu'] = 0
hands_drops['Te'] = 0
hands_drops['alpha'] = 0.5
hands_drops['beta'] = 0.5
# # с 5 по 20 декабря 2019
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(15/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(19/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(10/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(10/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(9/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(2/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(24/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(14/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(12/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(10/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(12/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(16/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(12/10))
# # пропуск  с июля по октябрь 2020
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(30/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(19/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(11/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(9/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(16/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(19/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(18/10))
# # март 2021
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(15/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(17/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(18/10))
# # апрель 2021
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(14/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(20/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(12/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(12/10))
# # # вышел на работу с сентября
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(11/10))
hands_drops = update_hands(hands_drops, 'гилан', 30/(21/12))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(15/10))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(17/10))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
hands_drops = update_hands(hands_drops, 'гилан', 30/(32/12))
hands_drops = update_hands(hands_drops, 'гилан', 30/(16/12))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(9/10))
hands_drops = update_hands(hands_drops, 'гилан', 30/(10/12))
hands_drops = update_hands(hands_drops, 'гилан ультра комфорт', 30/(11/12))
hands_drops = update_hands(hands_drops, 'гилан', 30/(12/12))

hands_drops

,name,mu,Te,alpha,beta
0,хилокомод,24.067828,6,3.5,78.429128
1,гилан,23.378571,5,3.0,195.451020
2,гилан ультра комфорт,32.727273,1,1.0,0.500000
3,стиллавит,0.000000,0,0.5,0.500000
4,артелак баланс,0.000000,0,0.5,0.500000
5,артелак всплеск уно,0.000000,0,0.5,0.500000
6,артелак всплеск,0.000000,0,0.5,0.500000
7,систейн ультра плюс,0.000000,0,0.5,0.500000
8,визмед,0.000000,0,0.5,0.500000
9,визмед лайт,0.000000,0,0.5,0.500000


In [12]:
tau_drops = gamma.rvs( a = hands_drops.alpha, scale=1/hands_drops.beta)
theta_drops = norm.rvs(hands_drops.mu, 1/hands_drops.Te)
hands_drops['tau'] = tau_drops
hands_drops['theta'] = theta_drops
hands_drops['SD'] = np.sqrt(1/tau_drops)
hands_drops['var95'] = theta_drops + norm.ppf(1-0.05/2) * hands_drops.SD


if hands_drops.mu.min() == 0:
    
    output2 = hands_drops.reindex(np.argsort(hands_drops.Te))
else:
    # hands_drops['sorter'] = 1/tau_drops + rho * theta_drops
    output2 = hands_drops.reindex(np.argsort(rho * theta_drops + 1/tau_drops))

output2

# todo этот вариант кода переписать как опцию максимизации
# hands_drops['var95'] = theta_drops + norm.ppf(0.05/2) * hands_drops.SD


# if hands_drops.mu.min() == 0:
#     output2 = hands_drops.reindex(np.argsort(hands_drops.Te))
# else:
#     output2 = hands_drops.reindex(np.argsort(rho * theta_drops - 1/tau_drops)[::-1])


,name,mu,Te,alpha,beta,tau,theta,SD,var95
Te,,,,,,,,,
3,стиллавит,0.000000,0,0.5,0.500000,4.108118,-inf,0.493377,-inf
4,артелак баланс,0.000000,0,0.5,0.500000,0.152603,inf,2.559870,inf
5,артелак всплеск уно,0.000000,0,0.5,0.500000,0.197944,inf,2.247652,inf
6,артелак всплеск,0.000000,0,0.5,0.500000,0.741338,inf,1.161427,inf
7,систейн ультра плюс,0.000000,0,0.5,0.500000,0.823417,inf,1.102022,inf
8,визмед,0.000000,0,0.5,0.500000,0.481515,inf,1.441104,inf
9,визмед лайт,0.000000,0,0.5,0.500000,3.667334,-inf,0.522185,-inf
10,хилабак,0.000000,0,0.5,0.500000,0.430981,inf,1.523250,inf
11,окутиарз,0.000000,0,0.5,0.500000,0.039051,-inf,5.060405,-inf


## для поездки с работы на ОТ, минимизуруем минуты

In [13]:
hands_back = pd.DataFrame({'name': ['2гис', 'яндекс карты', 'гугл карты', 'citymapper',
                                     'транспорт москвы'
                                     ]})
hands_back['mu'] = 0
hands_back['Te'] = 0
hands_back['alpha'] = 0.5
hands_back['beta'] = 0.5
hands_back = update_hands(hands_back, '2гис', '00:55:54')
hands_back = update_hands(hands_back, 'яндекс карты', '00:49:41')


hands_back

,name,mu,Te,alpha,beta
0,2гис,55.900000,1,1.0,0.5
1,яндекс карты,49.683333,1,1.0,0.5
2,гугл карты,0.000000,0,0.5,0.5
3,citymapper,0.000000,0,0.5,0.5
4,транспорт москвы,0.000000,0,0.5,0.5


In [14]:
tau_back = gamma.rvs( a = hands_back.alpha, scale=1/hands_back.beta)
theta_back = norm.rvs(hands_back.mu, 1/hands_back.Te)
hands_back['tau'] = tau_back
hands_back['theta'] = theta_back
hands_back['SD'] = np.sqrt(1/tau_back)
hands_back['var95'] = theta_back + norm.ppf(1-0.05/2) * hands_back.SD


if hands_back.mu.min() == 0:
    output3 = hands_back.reindex(np.argsort(hands_back.Te))
else:
    output3 = hands_back.reindex(np.argsort(rho * theta_back + 1/tau_back))

output3


,name,mu,Te,alpha,beta,tau,theta,SD,var95
Te,,,,,,,,,
2,гугл карты,0.000000,0,0.5,0.5,0.006776,inf,12.148164,inf
3,citymapper,0.000000,0,0.5,0.5,0.121498,-inf,2.868901,-inf
4,транспорт москвы,0.000000,0,0.5,0.5,0.468928,inf,1.460316,inf
0,2гис,55.900000,1,1.0,0.5,0.436314,57.613647,1.513912,60.580860
1,яндекс карты,49.683333,1,1.0,0.5,2.198258,50.436508,0.674467,51.758439
